In [1]:
from ipywidgets import FloatSlider, ValueWidget
import ipywidgets as widgets

from sidepanel import SidePanel

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters import * 

In [2]:
data = io.load('data/gauss4.p')
# data = io.load('data/simulations.p')

In [3]:
from ipywidgets import Output
debug = Output(title='debug')
debug

Output()

### Helper functions

#### view

In [4]:
def update_view(view, f):
    def _inner():
        view.show = filter_tree(view.tree, f)
    return _inner 

In [5]:
def view(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=update_view(v, f))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

#### reduce

In [6]:
def reduce_view(widget, tree, f):
    @debug.capture()
    def _inner():
        widget.ref = reduce_tree(tree, filter=f)
    return _inner

In [7]:
def reduce(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data.tree, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=reduce_view(v, data.tree, f))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

### Initial tree

In [8]:
tw = TreeWidget(data.tree)

In [9]:
v1, f1, p1 = view(tw, title='Initial Tree')

In [10]:
v2, f2, p2 = view(tw, attr='fitness', title='fitness')

/Users/yarden/miniconda3/envs/neup/lib/python3.7/site-packages/scikit_learn-0.19.2-py3.7-macosx-10.7-x86_64.egg/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


### Reduce tree

In [11]:
v2, r2, p2 = reduce(tw, attr='span', title='reduce')

In [12]:
v2.attr = 'parent_fitness'

### fitness

In [13]:
v2

TreeView(field='parent_fitness', tree_model=TreeWidget(attrs={'span': {0: 0.5392374574080602, 1: 0.00980789566…

#### test

In [14]:
tw2 = TreeWidget(data.tree)

In [15]:
f = AttrFilter(attr='span', func=lambda x, v: v <= x)

In [16]:
@debug.capture()
def test():
    print('test')
    tw2.ref = reduce_tree(tw.tree, filter=f)

In [17]:
m = Monitor(f, func=test)

In [18]:
test_view = TreeView(tw2, attr='span') 

In [19]:
p = SidePanel(title='Test')
with p:
    display(test_view, f)

In [20]:
fitness = view(tw2, attr='fitness', title='fitness')

In [21]:
parent_fitness = view(tw2, attr='parent_fitness', title='parent_fitness')

In [22]:
f_view = TreeView(tw2, attr='fitness')

In [23]:
f_view

TreeView(field='fitness', tree_model=TreeWidget(attrs={'span': {0: 0.5392374574080602, 1: 0.00980789566443424,…

In [24]:
test_view

TreeView(field='span', tree_model=TreeWidget(attrs={'span': {0: 0.5392374574080602, 1: 0.00980789566443424, 2:…

In [25]:
v1

TreeView(field='span', tree_model=TreeWidget(attrs={'span': {0: 0.5392374574080602, 1: 0.00980789566443424, 2:…